In [5]:
import os
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, month, dayofmonth, to_timestamp, when, lit, udf, round as spark_round
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.feature import VectorAssembler

# -------------------------
# 1. Initialize Spark
# -------------------------
spark = SparkSession.builder \
    .appName("ForestFire_Store_ALL_Data") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.mongodb.spark:mongo-spark-connector_2.12:10.4.0") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# -------------------------
# 2. Load Model
# -------------------------
model_path = "hdfs://namenode:9000/models/turkey_fire_model_v1"
try:
    fire_model = RandomForestClassificationModel.load(model_path)
    print("Model Loaded Successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    spark.stop()
    raise SystemExit

# -------------------------
# 3. Read Kafka
# -------------------------
json_schema = StructType([
    StructField("sensor_id", StringType()),
    StructField("timestamp", StringType()),
    StructField("temp", DoubleType()),
    StructField("humidity", DoubleType()),
    StructField("wind", DoubleType()),
    StructField("coord_x", DoubleType()),
    StructField("coord_y", DoubleType())
])

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "forest-sensors") \
    .option("startingOffsets", "latest") \
    .load()

df_raw = kafka_stream.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), json_schema).alias("data")) \
    .select("data.*")

# -------------------------
# 4. Feature Engineering (CRITICAL FIX: Grid Snapping)
# -------------------------
grid_step = 0.1  # same value used during training

df_features = (
    df_raw
    .withColumn("ts_obj", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))
    .withColumn("month", month(col("ts_obj")))
    .withColumn("day", dayofmonth(col("ts_obj")))
    # --- adjust coordinates to match training grid ---
    .withColumn("lat_grid", (spark_round(col("coord_x") / grid_step) * grid_step).cast("double"))
    .withColumn("lon_grid", (spark_round(col("coord_y") / grid_step) * grid_step).cast("double"))
    .withColumnRenamed("temp", "temp_max")
    .withColumnRenamed("wind", "wind_speed_max")
    .withColumnRenamed("humidity", "humidity_min")
    .withColumn("temp_mean", col("temp_max") - 2)
    .withColumn("temp_min", col("temp_max") - 5)
    .withColumn("humidity_mean", col("humidity_min") + 5)
    .withColumn("wind_speed_mean", col("wind_speed_max") * 0.8)
)

required_features = [
    'lat_grid', 'lon_grid', 'month', 'day',
    'temp_mean', 'temp_max', 'temp_min',
    'humidity_mean', 'humidity_min',
    'wind_speed_mean', 'wind_speed_max'
]

assembler = VectorAssembler(inputCols=required_features, outputCol="features", handleInvalid="keep")
df_final_input = assembler.transform(df_features)

# -------------------------
# 5. Prediction
# -------------------------
predictions = fire_model.transform(df_final_input)

# -------------------------
# 6. Extract Probability
# -------------------------
def extract_prob(v):
    try:
        return float(v[1])
    except:
        return 0.0

get_prob_udf = udf(extract_prob, DoubleType())

predictions = predictions.withColumn("fire_prob", get_prob_udf(col("probability")))

# -------------------------
# 7. Alert Logic (Hybrid: Model + Safety Net)
# -------------------------
alerts_df = predictions.select(
    col("sensor_id"), col("timestamp"), col("lat_grid"), col("lon_grid"),
    col("temp_max"), col("humidity_min"), col("fire_prob")
).withColumn("alert_status",
    when(col("fire_prob") >= 0.50, "CRITICAL FIRE")
    .when((col("fire_prob") >= 0.30) & (col("temp_max") > 30), "HIGH RISK ALERT")
    .when((col("temp_max") >= 40) & (col("humidity_min") < 25), "CRITICAL FIRE (Extreme Weather)")
    .otherwise("OK")
)

# -------------------------
# 8. Output to Console & MongoDB
# -------------------------
def process_all_data(df, epoch_id):
    # Use rdd.isEmpty() to check for empty DataFrame in batch context
    if df.rdd.isEmpty():
        return

    print(f"\n--- BATCH {epoch_id} ---")
    
    df_clean = df.select(
        col("timestamp").cast("string"),
        col("sensor_id").cast("string"),
        col("alert_status").cast("string"),
        col("temp_max").cast("double"),
        col("humidity_min").cast("double"),
        col("lat_grid").cast("double"),
        col("lon_grid").cast("double"),
        col("fire_prob").cast("double")
    )
    
    # display data
    df_clean.show(10, truncate=False)
    
    # save to MongoDB
    try:
        df_clean.write \
        .format("mongodb") \
        .mode("append") \
        .option("spark.mongodb.connection.uri", "mongodb://mongodb:27017") \
        .option("spark.mongodb.database", "bigdata") \
        .option("spark.mongodb.collection", "all_sensor_readings") \
        .save()
        print(f"Stored {df_clean.count()} records.")
    except Exception as e:
        print(f"Mongo Error: {e}")
        
    print(f"-----------------------\n")

print("Monitoring started... Expecting Alerts for SUMMER data...")

query = alerts_df.writeStream \
    .foreachBatch(process_all_data) \
    .start()

query.awaitTermination()


Model Loaded Successfully!
Monitoring started... Expecting Alerts for SUMMER data...

--- BATCH 27 ---

--- BATCH 1 ---
+-------------------+-----------+------------+--------+------------+--------+--------+-------------------+
|timestamp          |sensor_id  |alert_status|temp_max|humidity_min|lat_grid|lon_grid|fire_prob          |
+-------------------+-----------+------------+--------+------------+--------+--------+-------------------+
|2025-03-24 14:48:12|TR_SENS_001|OK          |23.36   |46.68       |38.0    |41.5    |0.14235104975437138|
+-------------------+-----------+------------+--------+------------+--------+--------+-------------------+

+-------------------+-----------+------------+--------+------------+--------+--------+-------------------+
|timestamp          |sensor_id  |alert_status|temp_max|humidity_min|lat_grid|lon_grid|fire_prob          |
+-------------------+-----------+------------+--------+------------+--------+--------+-------------------+
|2025-03-24 14:48:12|TR

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


Stored 19 records.
-----------------------

+-------------------+-----------+-------------------------------+--------+------------+--------+--------+-------------------+
|timestamp          |sensor_id  |alert_status                   |temp_max|humidity_min|lat_grid|lon_grid|fire_prob          |
+-------------------+-----------+-------------------------------+--------+------------+--------+--------+-------------------+
|2025-07-07 14:47:19|TR_SENS_012|OK                             |32.06   |37.09       |38.0    |33.5    |0.21166109492223267|
|2025-11-21 16:38:46|TR_SENS_015|OK                             |23.68   |50.31       |38.0    |33.5    |0.37053268878094786|
|2025-06-09 11:34:59|TR_SENS_008|CRITICAL FIRE (Extreme Weather)|40.32   |16.42       |38.0    |33.5    |0.20834638180712012|
|2025-03-28 14:02:09|TR_SENS_008|OK                             |17.79   |46.2        |38.0    |41.5    |0.11147748067704524|
|2025-06-22 16:58:02|TR_SENS_011|OK                             |29.95   |

KeyboardInterrupt: 

Stored 14 records.
-----------------------

Stored 8 records.
-----------------------


--- BATCH 40 ---

--- BATCH 14 ---

--- BATCH 33 ---
+-------------------+-----------+---------------+--------+------------+--------+--------+-------------------+
|timestamp          |sensor_id  |alert_status   |temp_max|humidity_min|lat_grid|lon_grid|fire_prob          |
+-------------------+-----------+---------------+--------+------------+--------+--------+-------------------+
|2025-06-22 16:58:02|TR_SENS_011|OK             |29.95   |35.54       |38.0    |41.5    |0.3665915901393977 |
|2025-05-06 11:19:57|TR_SENS_011|OK             |17.15   |49.78       |38.0    |41.5    |0.13069026085410218|
|2025-11-01 15:09:56|TR_SENS_017|OK             |27.01   |54.85       |38.0    |33.5    |0.1539486830057474 |
|2025-03-10 10:41:13|TR_SENS_020|OK             |17.53   |58.17       |38.0    |33.5    |0.04726670590345573|
|2025-06-09 14:41:21|TR_SENS_014|OK             |34.91   |35.04       |38.0    |33.5    |

In [8]:
count = collection.count_documents({})
print(f"Number of documents: {count}")

Number of documents: 263


In [12]:
from pymongo import MongoClient

# 1. الاتصال بقاعدة البيانات
# تأكد أن الرابط هو نفسه المستخدم في مشروعك
client = MongoClient("mongodb://mongodb:27017/")

# 2. اختيار قاعدة البيانات والجدول (Collection)
db = client["bigdata"]
collection = db["all_sensor_readings"]

# 3. معرفة عدد المستندات قبل الحذف (اختياري)
print(f"Count before delete: {collection.count_documents({})}")

# 4. حذف جميع البيانات
result = collection.delete_many({})

print(f"Deleted {result.deleted_count} documents.")
print("MongoDB is now CLEAN and ready for new data! 🧹✨")

Count before delete: 2496
Deleted 2496 documents.
MongoDB is now CLEAN and ready for new data! 🧹✨
